In [1]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
from datetime import datetime
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
# 모델, 스케일러 로드
model = joblib.load('./model/random_forest_model.joblib')
scaler = joblib.load('./model/scaler_rf.joblib')

In [3]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c) # (아날로그 → 디지털 변환기) 객체
ads.gain = 1 # 증폭 배율 설정

# ADS1115 채널 선택
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

ValueError: No I2C device at address: 0x48

In [ ]:
# 보정 파라미터
ZMPT_offset = 1.65  # 중심값
ZMPT_scale = 57.14  # 스케일 조정
ACS712_offset = 2.5
ACS712_sensitivity = 0.1

In [ ]:
# 파생 피처 생성 함수
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
# 후처리 함수 (8개 이상 연속 감지)
def postprocess_anomalies(y_bin, min_consecutive=8):
    if len(y_bin) < min_consecutive: # y_bin(예측값)의 길이가 연속적으로 8개보다 적다면
        return 0
    return int(sum(y_bin[-min_consecutive:]) >= min_consecutive) # y_bin의 인덱스 합값이 8 이상인 경우 True -> 1 반환

In [ ]:
# 실시간 센서 읽기 및 예측
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    data_buffer = []
    anomaly_buffer = []
    ANOMALY_THRESHOLD = 8
    first_anomaly_detected = False # 첫번째 이상 탐지 신호
    start_time = time.perf_counter() # 하드웨어 정밀 시간 측정 start

    while True:
        # 센서 읽기
        v_raw = voltage_ch.voltage 
        c_raw = current_ch.voltage

        # 센서값 보정
        voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        current = (c_raw - ACS712_offset) / ACS712_sensitivity

        data_buffer.append((voltage, current)) # 튜플 형태로 voltage, current를 data_buffer에 저장

        if len(data_buffer) >= 8: # data_buffer의 길이가 8 이상인 경우 (즉, 센서값이 8개 이상을 가진 경우)
            voltage_seq, current_seq = zip(*data_buffer[-8:]) # 언패킹(하나하나) 튜플 형태로 최근 값 8개를 반환하고 zip을 통해 같은 열에 있는 것을 가로로 묶음
            features = add_features(voltage_seq, current_seq) # 언패킹해서 분류한 전압 시퀀스와 전류 시퀀스를 이용해 특성 공학 진행
            features_scaled = scaler.transform(np.array(features).reshape(1, -1)) # 모델 학습을 위해서 표준화 작업(입력을 맞춤)

            y_pred = model.predict(features_scaled)
            anomaly = int(y_pred.flatten()) # predict() 결과가 기본적으로 배열 형태로 나오게 됨 (ex: [[0]]) → flatten으로 1D 배열로 평탄환 작업 또한 int()를 통해 깔끔하게 0 or 1 단일 숫자로 변환

            anomaly_buffer.append(anomaly) # 하나의 신호에 대한 이상 감지 여부를 anomaly_buffer에 저장
            if len(anomaly_buffer) > ANOMALY_THRESHOLD: # anomaly_buffer의 갯수가 8개 초과인 경우
                anomaly_buffer.pop(0) # 가장 앞에 있는 값을 방출

            if postprocess_anomalies(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD): # anomaly_buffer에 8개 모두가 이상을 감지한다면
                print("⚡ 아크 이상 감지!")
                if not first_anomaly_detected: # 첫번째 감지인지 여부
                    elapsed_time = time.perf_counter() - start_time
                    print(f"⏱️ 첫 번째 이상 탐지까지 걸린 시간: {elapsed_time:.6f}초")
                    first_anomaly_detected = True

        # 리소스 사용량 출력
        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리 사용량: {memory_usage:.2f} MB, 🧮 CPU 사용률: {cpu_usage:.2f}%")

        time.sleep(0.0001)  # 10kHz 샘플링

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")
    pass